In [1]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import os
import ipynbname

# Set Plotly template for better aesthetics
import plotly.io as pio
pio.templates.default = "plotly_white"

In [2]:
# Read all CSV files
print ()
# Get the directory where the notebook is located
notebook_path = ipynbname.path()
notebook_dir = notebook_path.parent
data_path = notebook_dir / 'data'
csv_files = list(data_path.rglob('*.csv')) if data_path.exists() else None
if not csv_files:
    raise FileNotFoundError(f"📂 Data folder not found: {data_path}")

print(f"In the {data_path} folder, there are {len(csv_files)} data files:")

for file in csv_files:
    print(f"- {file.name}")


# Load all dataframes
dataframes = {}
for file in csv_files:

    df = pd.read_csv(file)
    # Ensure we have a time column
    if 't' not in df.columns:
        df['t'] = range(len(df))
    dataframes[file.name] = df





In the C:\Users\Pavel Gromovikov\AppData\Roaming\JetBrains\DataSpell2025.1\projects\workspace\data folder, there are 5 data files:
- glutaugenpilz_log_20250712_194311.csv
- glutaugenpilz_log_20250712_220225.csv
- glutaugenpilz_log_20250713_112944.csv
- mycro_pure_in_A4_blow_into_SFXL_20250707_193210.csv
- mycro_pure_in_A4_blow_into_SFXL_20250712_142013.csv


In [3]:
#tidy up data - Tdie or Ppkg == 0 means there was a measurement error
for name, df in dataframes.items():
    # Filter out rows where Tdie or Ppkg equals 0 (measurement errors)
    df_filtered = df[(df['Tdie'] != 0) & (df['Ppkg'] != 0)]
    # Replace the original dataframe with the filtered one
    dataframes[name] = df_filtered
    print(f"Removed {len(df) - len(df_filtered)} measurement errors from {name}")


Removed 0 measurement errors from glutaugenpilz_log_20250712_194311.csv
Removed 1 measurement errors from glutaugenpilz_log_20250712_220225.csv
Removed 1 measurement errors from glutaugenpilz_log_20250713_112944.csv
Removed 1 measurement errors from mycro_pure_in_A4_blow_into_SFXL_20250707_193210.csv
Removed 4 measurement errors from mycro_pure_in_A4_blow_into_SFXL_20250712_142013.csv


In [4]:
# Function to create time series plots
def plot_time_series(df, title):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Temperature traces
    fig.add_trace(
        go.Scatter(x=df['t'], y=df['Tdie'], name="Tdie",
                   line=dict(color='#ff7f0e', width=2)),
        secondary_y=False
    )

    if 'Tair_in' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['t'], y=df['Tair_in'], name="Tair_in",
                       line=dict(color='#2ca02c', width=2)),
            secondary_y=False
        )

    if 'Tair_out' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['t'], y=df['Tair_out'], name="Tair_out",
                       line=dict(color='#d62728', width=2)),
            secondary_y=False
        )

    if 'Tpsu1' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['t'], y=df['Tpsu1'], name="Tpsu1",
                       line=dict(color='#9467bd', width=2)),
            secondary_y=False
        )

    if 'Tpsu2' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['t'], y=df['Tpsu2'], name="Tpsu2",
                       line=dict(color='#8c564b', width=2)),
            secondary_y=False
        )

    # Power trace
    fig.add_trace(
        go.Scatter(x=df['t'], y=df['Ppkg'], name="Ppkg",
                   line=dict(color='#1f77b4', width=2)),
        secondary_y=True
    )

    fig.update_layout(
        title=dict(text=title, x=0.5),
        plot_bgcolor='white',
        hovermode='x unified',
        width=1000,
        height=500,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )

    fig.update_xaxes(title_text="Time", gridcolor='lightgray')
    fig.update_yaxes(title_text="Temperature (°C)", secondary_y=False, gridcolor='lightgray')
    fig.update_yaxes(title_text="Package Power (W)", secondary_y=True, gridcolor='blue')

    return fig


# Plot time series for each dataset
for name, df in dataframes.items():
    fig = plot_time_series(df, f"Temperature and Power vs Time - {name}")
    fig.show()


In [5]:
# Function to create Tdie vs Ppkg plots
def plot_equilibrium_points(dataframes):
    # List to store equilibrium points
    equilibrium_points = []

    for name, df in dataframes.items():
        # Calculate power changes between consecutive points
        power_changes = df['Ppkg'].diff()

        # Find significant power jumps (>8W to catch ~10W steps)
        jump_points = power_changes[power_changes > 8].index

        # For each jump point, analyze the previous 60 points
        for jump in jump_points:
            if jump > 60:  # Make sure we have enough previous points
                # Get the 60 points before the jump
                prev_points = df.iloc[jump - 60:jump]

                # Calculate averages for this equilibrium period
                avg_temp = prev_points['Tdie'].mean()
                avg_power = prev_points['Ppkg'].mean()

                equilibrium_points.append({
                    'configuration': name,
                    'avg_power': avg_power,
                    'avg_temp': avg_temp
                })
    # Create dataframe from equilibrium points
    eq_df = pd.DataFrame(equilibrium_points)

    # Create line plot with markers
    fig = px.line(
        eq_df,
        x='avg_power',
        y='avg_temp',
        color='configuration',
        title='Equilibrium Temperature vs Power',
        labels={
            'avg_power': 'Average Package Power (W)',
            'avg_temp': 'Average Temperature (°C)',
            'configuration': 'Configuration'
        }
    )

    fig.update_traces(line=dict(width=1), mode='lines+markers', marker=dict(symbol='x', size=8))

    fig.update_layout(
        plot_bgcolor='white',
        width=1000,
        height=600,
        title_x=0.5,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    fig.update_xaxes(gridcolor='lightgray')
    fig.update_yaxes(gridcolor='lightgray')

    fig.show()



# Call the function to actually plot
plot_equilibrium_points(dataframes)


In [6]:
# Create combined plot
combined_data = pd.concat([
    df.assign(configuration=name)
    for name, df in dataframes.items()
])

fig = px.scatter(
    combined_data,
    x='Ppkg',
    y='Tdie',
    color='configuration',
    title='Temperature vs Power - All Configurations',
    labels={
        'Ppkg': 'Package Power (W)',
        'Tdie': 'Temperature (°C)',
        'configuration': 'Configuration'
    }
)

fig.update_traces(
    marker=dict(size=6, opacity=0.6),
    selector=dict(mode='markers')
)

fig.update_layout(
    plot_bgcolor='white',
    width=1200,
    height=800,
    title_x=0.5,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig.update_xaxes(gridcolor='lightgray')
fig.update_yaxes(gridcolor='lightgray')
fig.show()